In [ ]:
#%% IMPORTING LIBRARIES

import numpy as np
import tensorflow as tf
import matplotlib.cm as cm
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
layer_names = [layer.name for layer in keras_model.layers]
layer_names

In [ ]:
#%% CREATING THE HEATMAP FROM THE LAYERS' ACTIVATIONS

img_size = (224, 224)
layer_names=[layer.name for layer in keras_model.layers]

last_conv_layer_name = 'conv2d_15'
classifier_layer_names = [
    'max_pooling2d_7',
    'flatten_3',
    'dense_9',
    'dropout_6',
    'dense_10',
    'dropout_7',
    'dense_11']

def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    
    #img_array = test_set[0][8]
    img_array = img_array.reshape(1,224,224,1)
    img_array = img_array[:1]
    
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1,2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap
#%% 

In [ ]:
#%% DEFINING A FUNCTION TO DISPLAY THE HEATMAP ON THE REAL IMAGE

def display(heatmap, img):
    heatmap = np.uint8(255 * heatmap)
    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[1]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    
    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.005 + img.reshape(224,224,1)  #img_array.reshape(224,224,1)
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img

In [ ]:
#%% OBSERVING THE RAW IMAGE, THE HEATMAP AND THE SUPERIMPOSED IMAGES TOGETHER

for img in images10:
    heatmap = make_gradcam_heatmap(img,
                                   keras_model,
                                   last_conv_layer_name,
                                   classifier_layer_names)
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize= (10,5))
    
    ax1.imshow(img.reshape(224,224), cmap = 'gray')
    ax1.set_title('Raw MRI image')
    ax2.matshow(heatmap)
    ax3.imshow(display(heatmap,img))
    ax3.set_title('Superimposed Activation Heatmap')